<h1 align='center'> Curso de Estadística Bayesiana<br> Taller de Computación Bayesiana<h3>Autor</h3>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

<h3>Fork</h3>

<h3>Referencias</h3>




<h2> Modelo Binomial</h2>

<h3> El problema</h3>

Vamos a generar una observación Binomial. Para ello primero generamos un parámetro $\mu$ a partir de una distibución Beta. Usaremos los hiperparámetros $\alpha=5$ y $\beta=2$. Capricho del profesor. Vamos

In [ ]:
# Import librerias
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# genera una semilla de los números aleatorios para hacer rpetible el ejercicio
np.random.seed(seed = 500) 

<h3> Simula el parámetro $\mu$</h3>

In [ ]:
alpha = 5
beta = 2
mu = stats.beta(alpha,beta).rvs(1)

In [ ]:
print("mu= ",mu)

Este es el parámetro de interés que intentaremos recuperar más adelante.

<h3> Simula la observación aleatoria binomial $w$</h3>
Vamos a suponer un experimento $\text{Binomial} (N,\mu)$, con $N= 100$.

In [ ]:
N =100
w = stats.binom(n=N, p=mu).rvs(1)

In [ ]:
print("w= ",w)

<h3> Calculo de la posterior</h3>

Vamos a suponer un apriori $\beta(1.2,1.2)$. NOs olvidamos de la distribiución usada para generar $\mu$, puesto en la practica se desconoce.

Veámos un gráfico

In [ ]:
x = np.linspace(0,1,100)
y = stats.beta(1.2,1.2).pdf(x)

fig, ax = plt.subplots(1, 1,sharex = True, sharey = True, figsize=(6,4))
ax.set_ylim(0,1.2)
ax.plot(x,y)
ax.plot(0, 0, label="$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(1.2, 1.2), alpha=0)
ax.legend(fontsize=12)
fig.suptitle('Gráficos de la apriori propuesta Beta(1.2,1.2)',fontsize=16)
plt.show()

Esta distribución es cercana a la uniformo $U(0,1)$. ¿Porque?

La distribución posterior es $\text{Gamma}(1.2 +w, 1.2 + N-w)=\text{Gamma}(70.2, 32.2)$

<h3> Estimaciones</h3>

<h4> Moda Posterior </h4>

In [ ]:
from scipy.optimize import minimize_scalar as minimize

def f(t,alpha,beta):
    return  -stats.beta(alpha,beta).pdf(t)

lower = 1e-3
upper = 1  - 1e-3 
mode_f = minimize(f, bounds =(lower,upper),args=( 70.2,32.2), method='bounded' )

In [ ]:
mode_f

In [ ]:
MAP = mode_f.x

<h4> Mediana Posterior </h4>

In [ ]:
MEAP = stats.beta(70.2,32.2).ppf(0.5)
MEAP
# o también
stats.beta(70.2,32.2).median()

<h4> Media a Posterior </h4>

In [ ]:
EAP = stats.beta(alpha,beta).mean()
EAP

<h4> Intervalo de credibilidad  del 95%</h4>

In [ ]:
i_cred_94 = stats.beta(70.2,32.2).ppf((0.03,0.97))
print("I.C. 94\%",i_cred_94)

<h4> Desviación estándar de la posterior</h4>

In [ ]:
desv= stats.beta(alpha,beta).std()
desv

<h4> Resumen Estadístico de la Distribución Posterior</h4>

|Moda |Mediana | Media |Desv.|i_3%|i_97%|
|---|---|---|---|---|---|
|0.689|0.687|0.714|0.16|0.597|0.768|

In [ ]:
x = np.linspace(0,1,100)
y = stats.beta(1.2,1.2).pdf(x)

fig, ax = plt.subplots(1, 2,sharex = True, figsize=(20,6))
ax[0].set_ylim(0,9)
ax[0].plot(x,y)
ax[0].plot(0, 0, label="A priori: \n$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(1.2, 1.2), alpha=0)
ax[0].legend(fontsize=12)

y = stats.beta(70.2,32.2).pdf(x)

ax[1].set_ylim(0,9)
ax[1].plot(x,y)
ax[1].plot(0, 0, label="Posterior: \n$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(70.2, 32.2), alpha=0)
ax[1].legend(fontsize=12)
fig.suptitle('Gráficos de la apriori y la posterior de $\mu$',fontsize=16)
plt.show()

<h2> Stan</h2>

In [ ]:
# diccionario de datos

w=69
datos_binom = {'N': N,
               'w': w,
               'a': 1.2,
               'b': 1.2}

In [ ]:
datos_binom

In [ ]:
binomial_code = """
data {
    int<lower=0> N; // number of esays
    int<lower=0,upper=N> w; // estimated treatment effects
    real<lower=0.0> a; 
    real<lower=0.0> b; 
}

parameters {
    real<lower=0.0001,upper=0.9999> mu;
}

model {
   mu ~ beta(a, b);
    w ~ binomial(N, mu);
}

generated quantities {
    real log_lik;
    int w_hat;
    log_lik = binomial_lpmf(w | N,mu);
    w_hat = binomial_rng(N, mu);
}
"""

In [ ]:
print(binomial_code)

<h4> Compila el modelo </h4>

In [ ]:
# importamos Stan
import pystan

In [ ]:
# compile the  model
bm = pystan.StanModel(model_code=binomial_code)

<h4> Extrae las muestras de la Posterior </h4>

In [ ]:
# extract the samples
samples = bm.sampling(data=datos_binom, iter=2000, chains=4)

In [ ]:
samples

<h4> Moda de la Posterior </h4>

In [ ]:
op = bm.optimizing(data=datos_binom)
op

<h4> Análisis Exploratorio Bayesiano </h4>

Usaremos ArviZ

In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import xarray


In [ ]:
az.style.use('arviz-darkgrid')

xarray.set_options(display_style="html");

stan_data = az.from_pystan(
    posterior=samples,
    posterior_predictive='w_hat',
    observed_data=['w'],
    log_likelihood={'w': 'log_lik'},
    )

In [ ]:
az.plot_density(stan_data, var_names=['mu']);

In [ ]:
az.plot_autocorr(stan_data, var_names=("mu"))

In [ ]:
az.plot_forest(stan_data, var_names=("mu"))

In [ ]:
az.plot_trace(stan_data, var_names=("mu"))

In [ ]:
az.plot_mcse(stan_data)

In [ ]:
az.summary(samples)

In [ ]:
az.plot_posterior(samples)